In [1]:
import numpy as np
import pandas as pd
import os 
import nltk
import gensim
import time

In [2]:
temp_df=pd.read_csv(r"C:\Users\admin\Desktop\IMDB Dataset.csv")
df=temp_df.iloc[:10000] # Only 10K records
df.sample(4)

,review,sentiment
9665,I'm a historian. This movie is so wrong it hur...,negative
1574,Rajkumar Santoshi Without Any Doubt Has Direct...,negative
4019,"""Love is a Many-Splendored Thing"" is set in Ho...",positive
9449,I am oh soooo glad I have not spent money to g...,negative


In [3]:
df.drop_duplicates(inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_7744\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


### Preprocessing


In [4]:
import re

def remove(rabindra):
    cleaned_text = re.sub(re.compile('<.*?>'), ' ', rabindra)
    return re.sub(' +', ' ', cleaned_text).strip()

### Stopword Removal

In [5]:
from nltk.corpus import stopwords
list1=stopwords.words('english')
df['review']=df['review'].apply(lambda x :[item for item in x.split() if item not in list1]).apply(lambda x:" ".join(x))


C:\Users\admin\AppData\Local\Temp\ipykernel_7744\3998995059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x :[item for item in x.split() if item not in list1]).apply(lambda x:" ".join(x))


### Lowercasing

In [6]:
df['review']=df['review'].apply(lambda x:x.lower())

C:\Users\admin\AppData\Local\Temp\ipykernel_7744\2697927567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:x.lower())


In [7]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought wonderful way spend time hot summer ...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. how anyone say ""good hockey movie""...",negative
9997,this movie bad movie. but watching endless ser...,negative
9998,this movie probably made entertain middle scho...,negative


In [8]:
x = df.iloc[:, 0:1]
y = df['sentiment']

print("DataFrame (x):")
print(x)
print("\nSeries (y):")
print(y)


DataFrame (x):
                                                 review
0     one reviewers mentioned watching 1 oz episode ...
1     a wonderful little production. <br /><br />the...
2     i thought wonderful way spend time hot summer ...
3     basically there's family little boy (jake) thi...
4     petter mattei's "love time money" visually stu...
...                                                 ...
9995  fun, entertaining movie wwii german spy (julie...
9996  give break. how anyone say "good hockey movie"...
9997  this movie bad movie. but watching endless ser...
9998  this movie probably made entertain middle scho...
9999  smashing film film-making. shows intense stran...

[9983 rows x 1 columns]

Series (y):
0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object


In [9]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [10]:
story=[]
for doc in df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [11]:
model = gensim.models.Word2Vec(
window=10,
min_count = 2)

In [12]:
model.build_vocab(story)

In [13]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(6157483, 6736470)

In [14]:
len(model.wv.index_to_key)

31846

### Create a Document Vector 

In [15]:
def document_vector(doc):
    #Remove out of vocabulary word
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [16]:
document_vector(df['review'].values[0])

array([-0.17925577,  0.3771049 , -0.30377927, -0.06523907,  0.07667696,
       -0.5305976 ,  0.32946247,  0.6931923 , -0.26687115, -0.30005574,
       -0.2054101 , -0.4356749 , -0.43364495, -0.15889864,  0.05173063,
       -0.06986058,  0.23395456, -0.41756096, -0.27386087, -0.5884268 ,
        0.17759001,  0.03349234,  0.4631172 , -0.3559377 , -0.15746476,
        0.19129942, -0.31526116, -0.2848193 , -0.27529383,  0.07082173,
        0.46666747,  0.22836865,  0.06945398, -0.39477533, -0.04026747,
        0.44086257, -0.13635194, -0.3201201 , -0.06943347, -0.43538117,
        0.4376166 , -0.52816933,  0.04722259, -0.1620713 ,  0.39811352,
       -0.11139425, -0.1854866 ,  0.11935574,  0.23703814,  0.38138333,
        0.19952519, -0.27951354,  0.08700392,  0.06680976, -0.32324323,
        0.18758225,  0.23670268, -0.01095422, -0.14752574,  0.21117833,
        0.15233415,  0.07763148, -0.16770233,  0.14308423, -0.51717615,
        0.3807433 , -0.0346625 ,  0.35797632, -0.48158324,  0.36

In [17]:
from tqdm import tqdm           #Learn about tqdm
x=[]
for doc in tqdm(df['review'].values):
    x.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [09:28<00:00, 17.56it/s]


#### Note:tqdm is a popular python library used to display progress bars.
####  example

In [18]:
l=range(100)
for i in tqdm(l):
    print(i)
    time.sleep(0.1)

  1%|▊                                                                                 | 1/100 [00:00<00:10,  9.80it/s]

0
1


  3%|██▍                                                                               | 3/100 [00:00<00:10,  9.31it/s]

2
3


  5%|████                                                                              | 5/100 [00:00<00:10,  9.18it/s]

...

 97%|██████████████████████████████████████████████████████████████████████████████▌  | 97/100 [00:10<00:00,  9.23it/s]

96
97


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:10<00:00,  9.26it/s]

98
99


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.14it/s]


In [19]:
x=np.array(x)
x.shape

(9983, 100)

In [20]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y= encoder.fit_transform(df['sentiment'])

In [21]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [23]:
mnb=GaussianNB()
mnb.fit(x_train,y_train)
y_pred=mnb.predict(x_test)
accuracy_score(y_test,y_pred)

0.7240861291937907